# A simple classification model using Keras with Cloud TPUs

This notebook demonstrates using Cloud TPUs in colab to build a simple classification model using iris dataset to predict the species of the flower. This model is using 4 input features ***(SepalLength, SepalWidth, PetalLength, PetalWidth)*** to determine one of these flower species ***(Setosa, Versicolor, Virginica)***.

**Advantages:**
*   GCP account is not compulsory which is a must pre-requisite for the models using TPUEstimator()
*   This tutorial gives a way to take your own data insteasd of using already loaded data into Keras.



**NOTE:** This tutorial is just for learning how to write a simple model using Keras. It should not be used for comparision with training on CPU's because we have very less data in this iris_data example.

###Imports

In [0]:
#  Copyright 2018 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,0
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""An Example of a classification model using Keras for the Iris dataset."""

import json
import os
import pandas as pd
import pprint
import tensorflow as tf
import time
import numpy as np
from tensorflow import keras


In [0]:
print(tf.__version__)

1.12.0-rc1


###Resolve TPU Address

In [0]:
use_tpu = True #@param {type:"boolean"}

if use_tpu:
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TF_MASTER=''

with tf.Session(TF_MASTER) as session:
  print ('List of devices:')
  pprint.pprint(session.list_devices())

List of devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 10756513128697293510),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 8185962120762337342),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 11983346911074778577),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 16727884038864215972),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10359765307369019206),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 1811244341424894577),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 4392405938190321287),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 3351626764135995345),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 14422639591175693592),
 _DeviceAttribute

###FLAGS used as model params

In [0]:
# Model specific parameters

# TPU address
tpu_address = TF_MASTER

# Number of epochs
epochs = 50

# Number of steps_per_epoch
steps_per_epoch = 20

# NOTE: Total number of training steps = Number of epochs * Number of steps_per_epochs

# Total number of evaluation steps. If '0', evaluation after training is skipped
eval_steps = 50

###Download training input data and define prediction input & output

In [0]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

PREDICTION_INPUT_DATA = {
    'SepalLength': [6.9, 5.1, 5.9, 6.0, 5.5, 6.2, 5.5, 6.3],
    'SepalWidth': [3.1, 3.3, 3.0, 3.4, 2.5, 2.9, 4.2, 2.8],
    'PetalLength': [5.4, 1.7, 4.2, 4.5, 4.0, 4.3, 1.4, 5.1],
    'PetalWidth': [2.1, 0.5, 1.5, 1.6, 1.3, 1.3, 0.2, 1.5],
}

PREDICTION_OUTPUT_DATA = ['Virginica', 'Setosa', 'Versicolor', 'Versicolor', 'Versicolor', 'Versicolor', 'Setosa', 'Virginica']

def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path

def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path, test_path = maybe_download()

    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0, dtype={'SepalLength': pd.np.float32,
        'SepalWidth': pd.np.float32, 'PetalLength': pd.np.float32, 'PetalWidth': pd.np.float32, 'Species': pd.np.int32})
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0, dtype={'SepalLength': pd.np.float32,
        'SepalWidth': pd.np.float32, 'PetalLength': pd.np.float32, 'PetalWidth': pd.np.float32, 'Species': pd.np.int32})
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)

###Define model (2 hidden layers with 10 neurons in each)

In [0]:
def get_model():
  return keras.Sequential([
    keras.layers.Dense(10, input_shape=(4,), activation=tf.nn.relu, name = "Dense_1"),
    keras.layers.Dense(10, activation=tf.nn.relu, name = "Dense_2"),
    keras.layers.Dense(3, activation=None, name = "logits"),
    keras.layers.Dense(3, activation=tf.nn.softmax, name = "softmax")
  ])

In [0]:
dnn_model = get_model()
dnn_model.compile(optimizer=tf.train.AdagradOptimizer(learning_rate=0.1), 
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_crossentropy'])

dnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
Dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
logits (Dense)               (None, 3)                 33        
_________________________________________________________________
softmax (Dense)              (None, 3)                 12        
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________


###Creating a TPU model from a Keras Model

In [0]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    dnn_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TF_MASTER)))

tpu_model.summary()

INFO:tensorflow:Querying Tensorflow master (grpc://10.75.13.138:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 10756513128697293510)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 8185962120762337342)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 11983346911074778577)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 16727884038864215972)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10359765307369019206)
INFO:tensorflow:*** Available Device: _DeviceA

###Training of the model on TPU

In [0]:
# Fetch the data
(train_x, train_y), (test_x, test_y) = load_data()

# Train the model
tpu_model.fit(
  train_x, train_y,
  steps_per_epoch = steps_per_epoch,
  epochs=epochs,
)

Epoch 1/50
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(1395,), dtype=tf.int32, name=u'core_id_30'), TensorSpec(shape=(1395, 4), dtype=tf.float32, name=u'Dense_1_input_30'), TensorSpec(shape=(1395, 1), dtype=tf.float32, name=u'softmax_target_70')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for Dense_1_input
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 1.81500887871 secs
INFO:tensorflow:Setting weights on TPU model.
20/20 [==============================] - 4s 200ms/step - loss: 0.8510 - sparse_categorical_crossentropy: 0.8510
Epoch 2/50
20/20 [==============================] - 0s 17ms/step - loss: 0.3791 - sparse_categorical_crossentropy: 0.3791
Epoch 3/50
20/20 [==============================] - 0s 18ms/step - loss: 0.2966 - sparse_categorical_crossentropy: 0.2966
Epoch 4/50
20/20 [==============================] - 0s 17ms/step - loss: 0.2180 - sparse_cat

###Evaluation of the model

In [0]:
tpu_model.evaluate(test_x, test_y,
    steps = eval_steps)

INFO:tensorflow:New input shapes; (re-)compiling: mode=eval (# of cores 8), [TensorSpec(shape=(3,), dtype=tf.int32, name=u'core_id_40'), TensorSpec(shape=(3, 4), dtype=tf.float32, name=u'Dense_1_input_30'), TensorSpec(shape=(3, 1), dtype=tf.float32, name=u'softmax_target_70')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for Dense_1_input
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 0.717143058777 secs
50/50 [==============================] - 1s 27ms/step


[0.06343143433332443, 0.06343143433332443]

###Save the model

In [0]:
tpu_model.save_weights('./DNN_TPU_1024.h5', overwrite=True)

INFO:tensorflow:Copying TPU weights to the CPU


##Prediction

###Prediction data

In [0]:
COLUMNS_NAME=['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
data = pd.DataFrame(PREDICTION_INPUT_DATA, columns=COLUMNS_NAME)
print(data)

   SepalLength  SepalWidth  PetalLength  PetalWidth
0          6.9         3.1          5.4         2.1
1          5.1         3.3          1.7         0.5
2          5.9         3.0          4.2         1.5
3          6.0         3.4          4.5         1.6
4          5.5         2.5          4.0         1.3
5          6.2         2.9          4.3         1.3
6          5.5         4.2          1.4         0.2
7          6.3         2.8          5.1         1.5


###Prediction on TPU

In [0]:
predictions = tpu_model.predict(data)
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
for pred_dict, expec in zip(predictions, PREDICTION_OUTPUT_DATA):
  class_index = np.argmax(pred_dict)
  class_probability = np.max(pred_dict)
  print(template.format(SPECIES[class_index], 100*class_probability, expec))

INFO:tensorflow:New input shapes; (re-)compiling: mode=infer (# of cores 8), [TensorSpec(shape=(1, 4), dtype=tf.float32, name=u'Dense_1_input_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for Dense_1_input
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 0.63742518425 secs

Prediction is "Virginica" (99.5%), expected "Virginica"

Prediction is "Setosa" (99.9%), expected "Setosa"

Prediction is "Versicolor" (99.9%), expected "Versicolor"

Prediction is "Versicolor" (99.8%), expected "Versicolor"

Prediction is "Versicolor" (99.8%), expected "Versicolor"

Prediction is "Versicolor" (100.0%), expected "Versicolor"

Prediction is "Setosa" (100.0%), expected "Setosa"

Prediction is "Versicolor" (52.6%), expected "Virginica"


###Prediction on CPU

In [0]:
cpu_model = tpu_model.sync_to_cpu()
cpu_predictions = cpu_model.predict(data)
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
for pred_dict, expec in zip(cpu_predictions, PREDICTION_OUTPUT_DATA):
  class_index = np.argmax(pred_dict)
  class_probability = np.max(pred_dict)
  print(template.format(SPECIES[class_index], 100*class_probability, expec))

INFO:tensorflow:Copying TPU weights to the CPU

Prediction is "Virginica" (99.5%), expected "Virginica"

Prediction is "Setosa" (99.9%), expected "Setosa"

Prediction is "Versicolor" (99.9%), expected "Versicolor"

Prediction is "Versicolor" (99.8%), expected "Versicolor"

Prediction is "Versicolor" (99.8%), expected "Versicolor"

Prediction is "Versicolor" (100.0%), expected "Versicolor"

Prediction is "Setosa" (100.0%), expected "Setosa"

Prediction is "Versicolor" (52.6%), expected "Virginica"
